# One Hot Recomendation

one-hot es una técnica utilizada en sistemas de recomendación y aprendizaje automático para representar datos categóricos. En lugar de asignar un número entero a cada categoría, se utiliza un vector binario en el que solo un bit es 1 (indicando la presencia de la categoría) y todos los demás bits son 0. Esto ayuda a evitar que el modelo interprete relaciones ordinales entre categorías que no existen.

### PARA FINALIZAR ESTE TEMA Y COMPLETAR LOS REQUERIMIENTOS FUNCIONALES DEL MISMO, SE SOLICITA SE LE DESARROLLEN LAS SIGUIENTES MEJORAS AL SISTEMA QUE YA SE HA AVANZADO EN PYTHON.

1. HACER QUE LA ENTRADA DE USUARIO SEA DE FORMA DINAMICA, ES DECIR, QUE NO SEA CONSTANTE, QUE SEA VARIABLE Y DE LA FACILIDAD HACER PRUEBAS SOLO CON EL ID DE USUARIO DE RATINGS.CSV. ENTRADA DEL PERFIL DEL USUARIO.

2. QUE LA RECOMENDACIÓN FINAL MUESTRE EL POSTER(IMAGEN) DE LA PELICULA, PUEDE USAR LA MEJOR ESTRATEGIA QUE VEA FACTIBLE Y SE PUEDE CARGA SOLO PARCIALMENTE ALGUNAS PELICULAS EN LA FUENTE DE DATOS PARA SU PRUEBA Y DEMOSTRACION. PARA ESTO, SE COMPARTE LINK DE DONDE SE PUEDEN VER ESTOS POSTERS Y PUEDA UTILIZARLOS SI ASI LO DECIDE. “MovieGenre.csv”


In [673]:
import pandas as pd # type: ignore
from IPython.display import HTML


## aqui selecciona que usuario va a darle la recomendacion mediante su id

In [674]:



# Función para convertir links en imágenes HTML
def make_clickable(val):
    return f'<img src="{val}" width="150"/>'

# Muestra el DataFrame como HTML necesita estar en columna Poster la URL de la imagen
def mostrar_imagen(dataframe):
    dataframe['Poster'] = dataframe['Poster'].apply(make_clickable)
    return HTML(dataframe.to_html(escape=False))

In [675]:
#leer datos
peliculas = pd.read_csv('../2 Extras/One hot recomend/movies_data_fix.csv')
rating=pd.read_csv('../2 Extras/One hot recomend/ratings_data_fix.csv')
imagenes = pd.read_csv('../2 Extras/One hot recomend/MovieGenre.csv', encoding='latin1')
mostrar_imagen(imagenes.head(1))

#exixten links que no son validos y no se pueden mostrar

C:\Users\josem\AppData\Local\Temp\ipykernel_8760\105751786.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Poster'] = dataframe['Poster'].apply(make_clickable)


,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,


In [676]:
#eliminar en anio del campo tilte   
def eliminar_anio(dataframe):
    dataframe['year'] = dataframe.title.str.extract('(\\(\\d\\d\\d\\d\\))',expand=False)
    #print("\nExtraer en anio con parentecis\n",peliculas['year'].head())
    dataframe['year']= dataframe.year.str.extract('(\\d\\d\\d\\d)',expand=False)
    #print("\nExtraer en anio sin parentesis\n",peliculas['year'].head())
    dataframe['title'] = dataframe.title.str.replace(r"\s+\(.*\)","",regex=True)
    #print("\nEliminar el anio de las peliculas\n",peliculas.head())
    dataframe['title'] = dataframe['title'].apply(lambda x: x.strip())

imagenes=imagenes.rename(columns={'Title':'title'})

eliminar_anio(peliculas)
eliminar_anio(imagenes)


print(peliculas.head())
mostrar_imagen(imagenes.head(1))


   movie_id                        title                        genres  year
0         1                    Toy Story   Adventure|Children's|Comedy  1995
1         2                      Jumanji  Adventure|Children's|Fantasy  1995
2         3             Grumpier Old Men                Comedy|Romance  1995
3         4            Waiting to Exhale                  Comedy|Drama  1995
4         5  Father of the Bride Part II                        Comedy  1995


C:\Users\josem\AppData\Local\Temp\ipykernel_8760\105751786.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Poster'] = dataframe['Poster'].apply(make_clickable)


,imdbId,Imdb Link,title,IMDB Score,Genre,Poster,year
0,114709,http://www.imdb.com/title/tt114709,Toy Story,8.3,Animation|Adventure|Comedy,,1995


In [677]:
#eliminamos columnas de imagen que no necesitamos
imagenes=imagenes.drop(columns=['imdbId','Imdb Link','IMDB Score','Genre','year'])
mostrar_imagen(imagenes.head(1))


C:\Users\josem\AppData\Local\Temp\ipykernel_8760\105751786.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Poster'] = dataframe['Poster'].apply(make_clickable)


,title,Poster
0,Toy Story,


In [678]:
#separar los generos por el caracter |
peliculas['genres'] = peliculas.genres.str.split('|')

peliculas_co=peliculas.copy()
peliculas_co.head()

,movie_id,title,genres,year
0,1,Toy Story,"[Adventure, Children's, Comedy]",1995
1,2,Jumanji,"[Adventure, Children's, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [679]:
for index, row in peliculas.iterrows():
    for genere in row['genres']:
        peliculas_co.at[index,genere]=1

peliculas_co=peliculas_co.fillna(0)
print("peliculas codificadas: ",peliculas_co.head())

peliculas codificadas:     movie_id                        title                            genres  \
0         1                    Toy Story   [Adventure, Children's, Comedy]   
1         2                      Jumanji  [Adventure, Children's, Fantasy]   
2         3             Grumpier Old Men                 [Comedy, Romance]   
3         4            Waiting to Exhale                   [Comedy, Drama]   
4         5  Father of the Bride Part II                          [Comedy]   

   year  Adventure  Children's  Comedy  Fantasy  Romance  Drama  ...  \
0  1995        1.0         1.0     1.0      0.0      0.0    0.0  ...   
1  1995        1.0         1.0     0.0      1.0      0.0    0.0  ...   
2  1995        0.0         0.0     1.0      0.0      1.0    0.0  ...   
3  1995        0.0         0.0     1.0      0.0      0.0    1.0  ...   
4  1995        0.0         0.0     1.0      0.0      0.0    0.0  ...   

   Thriller  Horror  Animation  Sci-Fi  Documentary  War  Musical  Mystery

In [680]:
peliculas_co.head()

,movie_id,title,genres,year,Adventure,Children's,Comedy,Fantasy,Romance,Drama,...,Thriller,Horror,Animation,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,Toy Story,"[Adventure, Children's, Comedy]",1995,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children's, Fantasy]",1995,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama]",1995,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [681]:
#mostrar el rating de las peliculas
print("rating:", rating.head())
#eliminar la columna timestamp
rating=rating.drop('timestamp',axis=1)
print("\nrating new estructur:\n", rating.head())


rating:    user_id  movie_id  rating  timestamp
0        1      1193       5  978300760
1        1       661       3  978302109
2        1       914       3  978301968
3        1      3408       4  978300275
4        1      2355       5  978824291

rating new estructur:
    user_id  movie_id  rating
0        1      1193       5
1        1       661       3
2        1       914       3
3        1      3408       4
4        1      2355       5


In [682]:
#unir las tablas de peliculas y rating por el campo movie_id
peliculas_rating=pd.merge(peliculas_co,rating,on='movie_id')
peliculas_rating=peliculas_rating.drop(columns=['genres','year','Adventure','Animation',"Children's",'Comedy','Fantasy','Romance','Drama','Action','Crime','Thriller','Horror','Mystery','Sci-Fi','War','Musical','Documentary','Western','Film-Noir', 'movie_id'])
print("\nTabla de peliculas con rating:\n",peliculas_rating.head())

#seleccionamos el usuario 
usuario_en=peliculas_rating[peliculas_rating['user_id']==usuario_id]
usuario_en=usuario_en.drop(columns=['user_id'])
print("\nTabla de peliculas con rating del usuario 1:\n",usuario_en.head())



Tabla de peliculas con rating:
        title  user_id  rating
0  Toy Story        1       5
1  Toy Story        6       4
2  Toy Story        8       4
3  Toy Story        9       5
4  Toy Story       10       5

Tabla de peliculas con rating del usuario 1:
                                      title  rating
0                                Toy Story       5
22893                           Pocahontas       5
41541                            Apollo 13       5
67447   Star Wars: Episode IV - A New Hope       4
141660                    Schindler's List       5


In [683]:
# #vamos a crear nuestro algoritmo de sugerencua, acorde el perfil de usuario
# usuario_en=[
#     {'title':'Toy Story','rating':3.5},
#     {'title':'Jumanji','rating':2},

#     ]

#CON EN ANTERIOR USUARIO YA NO ES NECESARIO CAMBIAR ALGO DE AQUI EN ADELANTE, MAS QUE EL ULTIMPO PARA QUE SE MUESTRE LAS IMAGENES

#lo convertimos en un dataframe
entrada_peli=pd.DataFrame(usuario_en)
print("peliculas usuario:\n", entrada_peli)



peliculas usuario:
                                                   title  rating
0                                             Toy Story       5
22893                                        Pocahontas       5
41541                                         Apollo 13       5
67447                Star Wars: Episode IV - A New Hope       4
141660                                 Schindler's List       5
144754                                    Secret Garden       4
158459                                          Aladdin       4
167921                  Snow White and the Seven Dwarfs       4
168684                             Beauty and the Beast       5
172011                                            Fargo       4
179963                        James and the Giant Peach       3
185725  Wallace & Gromit: The Best of Aardman Animation       3
190432                                      Close Shave       3
196749                          Hunchback of Notre Dame       4
222307              

In [684]:
#agregamos el movie_id a las peliculas del usuario
Id = peliculas[peliculas['title'].isin(entrada_peli['title'].tolist())]
entrada_peli=pd.merge(Id, entrada_peli,how='inner',on=None, validate='many_to_many')
print("peliculas usuario con id:\n", entrada_peli)

peliculas usuario con id:
     movie_id                                            title  \
0          1                                        Toy Story   
1         48                                       Pocahontas   
2        150                                        Apollo 13   
3        260               Star Wars: Episode IV - A New Hope   
4        277                           Miracle on 34th Street   
5        527                                 Schindler's List   
6        531                                    Secret Garden   
7        588                                          Aladdin   
8        594                  Snow White and the Seven Dwarfs   
9        595                             Beauty and the Beast   
10       608                                            Fargo   
11       661                        James and the Giant Peach   
12       720  Wallace & Gromit: The Best of Aardman Animation   
13       745                                      Close Shave  

In [685]:
entrada_peli.head()

,movie_id,title,genres,year,rating
0,1,Toy Story,"[Adventure, Children's, Comedy]",1995,5
1,48,Pocahontas,"[Animation, Children's, Musical, Romance]",1995,5
2,150,Apollo 13,[Drama],1995,5
3,260,Star Wars: Episode IV - A New Hope,"[Action, Adventure, Fantasy, Sci-Fi]",1977,4
4,277,Miracle on 34th Street,[Drama],1994,4


In [686]:
#eliminamos las columnas que no necesitamos
entrada_peli=entrada_peli.drop('genres',axis=1).drop('year',axis=1)
entrada_peli.head()

,movie_id,title,rating
0,1,Toy Story,5
1,48,Pocahontas,5
2,150,Apollo 13,5
3,260,Star Wars: Episode IV - A New Hope,4
4,277,Miracle on 34th Street,4


In [687]:
#codicar pe;iculas del usuario para el metodo one hot encoding
peli_usuario=peliculas_co[peliculas_co['movie_id'].isin(entrada_peli['movie_id'].tolist())]
peli_usuario.head()

,movie_id,title,genres,year,Adventure,Children's,Comedy,Fantasy,Romance,Drama,...,Thriller,Horror,Animation,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,Toy Story,"[Adventure, Children's, Comedy]",1995,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,48,Pocahontas,"[Animation, Children's, Musical, Romance]",1995,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
148,150,Apollo 13,[Drama],1995,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,260,Star Wars: Episode IV - A New Hope,"[Action, Adventure, Fantasy, Sci-Fi]",1977,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
274,277,Miracle on 34th Street,[Drama],1994,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [688]:
#continuamos con el one hot encoding y ajustes de datos
peli_usuario=peli_usuario.reset_index(drop=True)
tabla_generos=peli_usuario.drop('movie_id',axis=1).drop('title',axis=1).drop('genres',axis=1).drop('year',axis=1)
print(tabla_generos.head()) 

   Adventure  Children's  Comedy  Fantasy  Romance  Drama  Action  Crime  \
0        1.0         1.0     1.0      0.0      0.0    0.0     0.0    0.0   
1        0.0         1.0     0.0      0.0      1.0    0.0     0.0    0.0   
2        0.0         0.0     0.0      0.0      0.0    1.0     0.0    0.0   
3        1.0         0.0     0.0      1.0      0.0    0.0     1.0    0.0   
4        0.0         0.0     0.0      0.0      0.0    1.0     0.0    0.0   

   Thriller  Horror  Animation  Sci-Fi  Documentary  War  Musical  Mystery  \
0       0.0     0.0        0.0     0.0          0.0  0.0      0.0      0.0   
1       0.0     0.0        1.0     0.0          0.0  0.0      1.0      0.0   
2       0.0     0.0        0.0     0.0          0.0  0.0      0.0      0.0   
3       0.0     0.0        0.0     1.0          0.0  0.0      0.0      0.0   
4       0.0     0.0        0.0     0.0          0.0  0.0      0.0      0.0   

   Film-Noir  Western  
0        0.0      0.0  
1        0.0      0.0  
2 

In [689]:
#creamos matriz de peso
print("tabla de generos transpuesta: \n",tabla_generos.T)
print("\n")
print("datos para ver el reating: \n",entrada_peli['rating'])

tabla de generos transpuesta: 
               0    1    2    3    4    5    6    7    8    9   ...   45   46  \
Adventure    1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
Children's   1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  ...  0.0  0.0   
Comedy       1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  1.0   
Fantasy      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
Romance      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
Drama        0.0  0.0  1.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  ...  0.0  0.0   
Action       0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
Crime        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
Thriller     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0   
Horror       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
Animation    0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  ...  0.0  0.0   
Sci-Fi  

In [690]:
#multiplicamos la tabla de generos por el rating de las peliculas
perfil_usuario=tabla_generos.T.dot(entrada_peli['rating'])
print("perfil de usuario: \n",perfil_usuario)

perfil de usuario: 
 Adventure       25.0
Children's      85.0
Comedy          58.0
Fantasy         12.0
Romance         22.0
Drama          101.0
Action          25.0
Crime            8.0
Thriller        11.0
Horror           0.0
Animation       69.0
Sci-Fi          13.0
Documentary      0.0
War             10.0
Musical         60.0
Mystery          0.0
Film-Noir        0.0
Western          0.0
dtype: float64


In [691]:
#mostramos los generos de la tabla original
generos=peliculas_co.set_index(peliculas_co['movie_id'])
generos.head()

,movie_id,title,genres,year,Adventure,Children's,Comedy,Fantasy,Romance,Drama,...,Thriller,Horror,Animation,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,1,Toy Story,"[Adventure, Children's, Comedy]",1995,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Jumanji,"[Adventure, Children's, Fantasy]",1995,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Waiting to Exhale,"[Comedy, Drama]",1995,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [692]:
#ajustar info de generos
generos=generos.drop('title',axis=1).drop('genres',axis=1).drop('year',axis=1)
print("generos ajustados: \n",generos.head())
generos.head()

generos ajustados: 
           movie_id  Adventure  Children's  Comedy  Fantasy  Romance  Drama  \
movie_id                                                                     
1                1        1.0         1.0     1.0      0.0      0.0    0.0   
2                2        1.0         1.0     0.0      1.0      0.0    0.0   
3                3        0.0         0.0     1.0      0.0      1.0    0.0   
4                4        0.0         0.0     1.0      0.0      0.0    1.0   
5                5        0.0         0.0     1.0      0.0      0.0    0.0   

          Action  Crime  Thriller  Horror  Animation  Sci-Fi  Documentary  \
movie_id                                                                    
1            0.0    0.0       0.0     0.0        0.0     0.0          0.0   
2            0.0    0.0       0.0     0.0        0.0     0.0          0.0   
3            0.0    0.0       0.0     0.0        0.0     0.0          0.0   
4            0.0    0.0       0.0     0.0      

,movie_id,Adventure,Children's,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Animation,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
movie_id,,,,,,,,,,,,,,,,,,,
1,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [693]:
#generamos la ponderacion
recom=((tabla_generos*perfil_usuario).sum(axis=1))/(perfil_usuario.sum())
print("recomendacion: \n",recom.head(50))
recom.shape

recomendacion: 
 0     0.336673
1     0.472946
2     0.202405
3     0.150301
4     0.202405
5     0.222445
6     0.372745
7     0.545090
8     0.428858
9     0.428858
10    0.240481
11    0.428858
12    0.138277
13    0.276553
14    0.428858
15    0.164329
16    0.543086
17    0.120240
18    0.428858
19    0.406814
20    0.428858
21    0.120240
22    0.422846
23    0.202405
24    0.260521
25    0.202405
26    0.202405
27    0.142285
28    0.302605
29    0.202405
30    0.595190
31    0.246493
32    0.202405
33    0.308617
34    0.202405
35    0.202405
36    0.308617
37    0.272545
38    0.308617
39    0.116232
40    0.044088
41    0.424850
42    0.202405
43    0.308617
44    0.110220
45    0.022044
46    0.116232
47    0.140281
48    0.318637
49    0.116232
dtype: float64


(55,)

In [694]:
#mostrar de manera mas amigable 
recom=pd.DataFrame(generos['movie_id'])

#multiplicamos los generos por la matriz de peso y tomar el merjor promedio
recom['ponderacion']=((generos*perfil_usuario).sum(axis=1))/(perfil_usuario.sum())
#recom=pd.DataFrame(recom)
recom.head(20)


,movie_id,ponderacion
movie_id,,
1,1,0.336673
2,2,0.244489
3,3,0.160321
4,4,0.318637
5,5,0.116232
6,6,0.088176
7,7,0.160321
8,8,0.220441
9,9,0.050100


In [695]:
#ordenar de mejor ponderacion 
recome=recom.sort_values('ponderacion',ascending=False)
recome.head(20)

,movie_id,ponderacion
movie_id,,
1566,1566,0.595190
2080,2080,0.589178
2081,2081,0.589178
2078,2078,0.545090
2102,2102,0.545090
588,588,0.545090
919,919,0.543086
2138,2138,0.535070
673,673,0.498998


In [696]:
#estrategia de visualizacion para el usuario
peliculas.set_index('movie_id')
recom.set_index('movie_id')
movies_df_positon=peliculas.join(recom, on='movie_id', lsuffix='_key', validate='one_to_one')
movies_df_positon=movies_df_positon.sort_values(by='ponderacion',ascending=False)
movies_df_positon=pd.merge(movies_df_positon,imagenes[['title', 'Poster']], on='title', how='left')
movies_df_positon = movies_df_positon.drop_duplicates(subset=['movie_id', 'title'], keep='first')
movies_df_positon.head(20)

,movie_id_key,title,genres,year,movie_id,ponderacion,Poster
0,1566,Hercules,"[Adventure, Animation, Children's, Comedy, Mus...",1997,1566,0.595190,https://images-na.ssl-images-amazon.com/images...
3,2080,Lady and the Tramp,"[Animation, Children's, Comedy, Musical, Romance]",1955,2080,0.589178,https://images-na.ssl-images-amazon.com/images...
4,2081,Little Mermaid,"[Animation, Children's, Comedy, Musical, Romance]",NaN,2081,0.589178,NaN
5,2078,Jungle Book,"[Animation, Children's, Comedy, Musical]",NaN,2078,0.545090,https://images-na.ssl-images-amazon.com/images...
6,2102,Steamboat Willie,"[Animation, Children's, Comedy, Musical]",1940,2102,0.545090,https://images-na.ssl-images-amazon.com/images...
7,588,Aladdin,"[Animation, Children's, Comedy, Musical]",1992,588,0.545090,https://images-na.ssl-images-amazon.com/images...
10,919,Wizard of Oz,"[Adventure, Children's, Drama, Musical]",NaN,919,0.543086,NaN
11,2138,Watership Down,"[Animation, Children's, Drama, Fantasy]",1978,2138,0.535070,https://images-na.ssl-images-amazon.com/images...
12,673,Space Jam,"[Adventure, Animation, Children's, Comedy, Fan...",1996,673,0.498998,https://images-na.ssl-images-amazon.com/images...
13,1812,Wide Awake,"[Children's, Comedy, Drama]",1998,1812,0.488978,https://images-na.ssl-images-amazon.com/images...


In [697]:

mostrar_imagen(movies_df_positon.head(20))

C:\Users\josem\AppData\Local\Temp\ipykernel_8760\105751786.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Poster'] = dataframe['Poster'].apply(make_clickable)


,movie_id_key,title,genres,year,movie_id,ponderacion,Poster
0,1566,Hercules,"[Adventure, Animation, Children's, Comedy, Musical]",1997,1566,0.595190,
3,2080,Lady and the Tramp,"[Animation, Children's, Comedy, Musical, Romance]",1955,2080,0.589178,
4,2081,Little Mermaid,"[Animation, Children's, Comedy, Musical, Romance]",NaN,2081,0.589178,
5,2078,Jungle Book,"[Animation, Children's, Comedy, Musical]",NaN,2078,0.545090,
6,2102,Steamboat Willie,"[Animation, Children's, Comedy, Musical]",1940,2102,0.545090,
7,588,Aladdin,"[Animation, Children's, Comedy, Musical]",1992,588,0.545090,
10,919,Wizard of Oz,"[Adventure, Children's, Drama, Musical]",NaN,919,0.543086,
11,2138,Watership Down,"[Animation, Children's, Drama, Fantasy]",1978,2138,0.535070,
12,673,Space Jam,"[Adventure, Animation, Children's, Comedy, Fantasy]",1996,673,0.498998,
13,1812,Wide Awake,"[Children's, Comedy, Drama]",1998,1812,0.488978,
